# Lesson 7

[Lesson 7](https://learn.deeplearning.ai/courses/langchain/lesson/7/agents) is All about agents. This is the line chain way of Enabling large language models to use tools.

 as a toy example, I'm going to Re-implement the calculator two different ways. First, we're going to implement a calculator By just using the pretty fine line chain functions. Afterwards were going to implement it from scratch, defining our own tools, just like in my blog posts [How to Turn GPT into a Calculator](https://chrwittm.github.io/posts/2024-07-30-llm-calculator1/)  - and maybe also in [Building the Apple Calculator in a Jupyter Notebook](https://chrwittm.github.io/posts/2024-08-02-llm-calculator2-vision/).



## Building a calculator with Langchain

In [7]:
import os
from dotenv import load_dotenv

load_dotenv() #contains the OPENAI_API_KEY

True

In [8]:
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [9]:
#!pip install numexpr

In [10]:
from langchain.agents import load_tools, initialize_agent, AgentType

tools = load_tools(["llm-math"], llm=llm)

agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

/var/folders/cs/g41bp1210qv6w4mt5z465x8c0000gn/T/ipykernel_30264/3335991520.py:5: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent= initialize_agent(


In [11]:
agent("What is ( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )")

/var/folders/cs/g41bp1210qv6w4mt5z465x8c0000gn/T/ipykernel_30264/2048359261.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent("What is ( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )")




> Entering new AgentExecutor chain...
Thought: I need to calculate the expression step by step. First, I will calculate the sum in the numerator and the product in the denominator, then perform the division, and finally calculate the last part of the expression before adding everything together.

Action:
```
{
  "action": "Calculator",
  "action_input": "( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )"
}
```

Observation: Answer: 2634.857142857143
Thought:I now know the final answer.
Final Answer: 2634.857142857143

> Finished chain.


{'input': 'What is ( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )',
 'output': '2634.857142857143'}

In [12]:
( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )

2634.857142857143

In [13]:
import langchain

langchain.debug=True
agent("What is ( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )")
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is ( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "What is ( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the \"action\" field are: Calculator\n\nThe $JSON_BLOB should only contain a SINGLE action, do NO

## Building a calculator from scratch 

Let's create a calculator from scratch, using the approach From my blog post [How to Turn GPT into a Calculator](https://chrwittm.github.io/posts/2024-07-30-llm-calculator1/), giving the LLM access to basic arithmetic.


In analogy to the lesson, I defined the tools like this.

In [14]:
from langchain.agents import tool

@tool
def add(a: float, b: float = 1.0) -> str:
    "Adds a + b"
    return str(a + b)

@tool
def subtract(a: float, b: float = 1.0) -> str:
    "Subtracts a - b"
    return str(a - b)

@tool
def multiply(a: float, b: float = 1.0) -> str:
    "Multiplies a * b"
    return str(a * b)

@tool
def divide(a: float, b: float = 1.0) -> str:
    "Divides a / b"
    if b == 0:
        return "Division by zero is not allowed."
    return str(a / b)

As it turned out, the agent type `` Only supports tools with one input parameter. Switching to another agent type solved the problem.

In [20]:
agent=initialize_agent(
    [add, subtract, multiply, divide], 
    llm, 
    #agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, #-> does not work
    #agent=AgentType.OPENAI_FUNCTIONS, #-> works
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, #-> works
    handle_parsing_errors=True,
    verbose = True)

In [21]:
agent("What is ( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )")



> Entering new AgentExecutor chain...
Thought: I need to break down the expression step by step. First, I will calculate the sum of 5647 and 3241, then the product of 7 and 2, and finally the sum of 657 and 343 multiplied by 2.

Action:
```
{
  "action": "add",
  "action_input": {
    "a": 5647,
    "b": 3241
  }
}
```

Observation: 8888.0
Thought:Now that I have the sum of 5647 and 3241, which is 8888, I will calculate the product of 7 and 2.

Action:
```
{
  "action": "multiply",
  "action_input": {
    "a": 7,
    "b": 2
  }
}
```

Observation: 14.0
Thought:Now that I have the product of 7 and 2, which is 14, I will divide the sum of 5647 and 3241 (which is 8888) by this product (14).

Action:
```
{
  "action": "divide",
  "action_input": {
    "a": 8888,
    "b": 14
  }
}
```


Observation: 634.8571428571429
Thought:Now that I have the result of the division, which is approximately 634.86, I will calculate the sum of 657 and 343, and then multiply that result by 2.

Action:
```
{

{'input': 'What is ( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )',
 'output': 'The result of the expression ( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 ) is approximately 2634.86.'}

In [18]:
( 5647 + 3241 ) / ( 7 * 2 ) + ( ( 657 + 343 ) * 2 )

2634.857142857143

The result is correct, and the choice of the right agent definitely matters, so what type of agents are there and how can we choose?

In [22]:
AgentType

<enum 'AgentType'>

In [23]:
print("Available Agent Types:")
for agent in AgentType:
    print(f"- {agent}")

Available Agent Types:
- zero-shot-react-description
- react-docstore
- self-ask-with-search
- conversational-react-description
- chat-zero-shot-react-description
- chat-conversational-react-description
- structured-chat-zero-shot-react-description
- openai-functions
- openai-multi-functions


In [24]:
help(AgentType)

Help on class AgentType in module langchain.agents.agent_types:

class AgentType(builtins.str, enum.Enum)
 |  AgentType(value, names=None, *, module=None, qualname=None, type=None, start=1)
 |  
 |  .. deprecated:: 0.1.0 Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.
 |  
 |  An enum for agent types.
 |  
 |  See documentation: https://python.langchain.com/docs/modules/agents/agent_types/
 |  
 |  Method resolution order:
 |      AgentType
 |      builtins.str
 |      enum.Enum
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  CHAT_CONVERSATIONAL_REACT_DESCRIPTION = <AgentType.CHAT_CONVERSATIONAL...
 |  
 |  CHAT_ZERO_SHOT_REACT_DESCRIPTION = <AgentType.CHAT_ZERO_SHOT_REACT_DES...
 |  
 |  CONVERSATIONAL_REACT_DESCRIPTION = <AgentType.CONVERSATIONAL_REACT_DES...
 |  
 |  OPENAI_FUNCTIONS = <AgentType.OPENAI_FUNCTIONS: 'openai-functions'>
 |  
 |  OPENAI_MULTI_FUNCTIONS = <AgentType.OPENAI_

Here is another example of Lang chain, moving fast: the Link in the documentation does not exist anymore.

Doing a quick search revealed: https://python.langchain.com/docs/how_to/agent_executor/, so there is a lot more to explore!

